# BI-Projekt
## Aufgabe 3

***3. Gibt es Turniere, bei denen die Deutschen besonders gut oder besonders schlecht
abschneiden? Wenn ja, ist ein regionales Muster oder eine andere Ursache zu erkennen?***

In [1]:
#Relevante module
import pandas as pd
import folium
from folium import plugins

Die atp_tourneys_till_2022 tabelle wurde angereichert mit Koordinaten. Problematisch sind aber hierbei Davis Cup Spiele. Aufgrund des hohen Aufwandes werden die Koordinaten für diese Leer gelassen und später auch entfernt.

In [2]:
df_tourney_coord = pd.read_csv('data/atp_tourneys_till_2022_coordinates.csv')
df_tourney_coord.sample(5)

,tourney_id,tourney_name,tourney_location,surface,draw_size,tourney_level,tourney_date,latitude,longitude
7212,2019-5014,Shanghai Masters,Shanghai,Hard,64,M,20191007,31.232276,121.469207
5086,1978-310,Florence,Florence,Clay,32,A,19780521,43.769871,11.255576
463,2016-0316,Bastad,Bastad,Clay,32,A,20160711,56.426790,12.860715
4559,2017-M-DC-2017-WG-M-BEL-FRA-01,Davis Cup WG F: BEL vs FRA,Davis Cup WG F: BEL vs FRA,Hard,4,D,20171124,NaN,NaN
6278,1989-421,Montreal / Toronto,Montreal / Toronto,Hard,56,M,19890814,45.448111,-73.639597


In [3]:
df_matches = pd.read_csv("data/atp_matches_till_2022.csv")
df_matches.sample(5)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
99805,1994-421,Canada Masters,Hard,56,M,19940725,33,102358,NaN,NaN,...,23.0,14.0,10.0,9.0,2.0,7.0,49.0,747.0,13.0,1850.0
72897,1987-404,Indian Wells,Hard,56,M,19870216,12,100934,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,67.0,NaN,66.0,NaN
19311,1973-416,Rome,Clay,128,M,19730602,42,100081,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156269,2011-741,Eastbourne,Grass,32,A,20110612,8,104731,6.0,NaN,...,37.0,32.0,10.0,10.0,4.0,6.0,38.0,1185.0,558.0,47.0
171447,2016-0319,Kitzbuhel,Clay,32,A,20160718,272,104999,NaN,NaN,...,58.0,38.0,11.0,12.0,0.0,2.0,146.0,398.0,88.0,657.0


Ebenso ist die atp_players_till_2022 erweitert worden um leichter auf das Land schließen zu können.

In [4]:
df_players = pd.read_csv('data/atp_players_till_2022_ioc2country_v2.csv', delimiter=';')

Die Fragestellung bezieht sich explizit auf die Deutschen. Als IOC code gibt es aber eine handvoll Einträge die relevanten sind neben GER, da die Daten auch soweit zurückgehen bis zur DDR/BRD, also dem geteilten Deutschland.  
Relevante IOC-Codes:  
EUA FRG GDR GER.

Als relevanter IOC-Code hat sich auch DEU herausgestellt in der player-Tabelle. DEU ist eigentlich kein echter Code und wurde vermutlich einfach nur falsch eingetragen.

In [5]:
df_players.loc[df_players.ioc.eq('DEU')]

,player_id,name_first,name_last,hand,dob,ioc,ioc_nation,height,wikidata_id
58036,211386,Alessio,Vasquez Gehrke,U,NaN,DEU,Germany,NaN,NaN
58037,211387,Jonas Pelle,Hartenstein,U,NaN,DEU,Germany,NaN,NaN
58146,211496,Philipp,Pavlenko,U,NaN,DEU,Germany,NaN,NaN
58675,212025,Yannik,Kelm,U,NaN,DEU,Germany,NaN,NaN


Für die Fragestellung werden nun alle relevanten Spieler gefiltert.

In [6]:
df_ger_players = df_players.loc[df_players.ioc.eq('GDR') | df_players.ioc.eq('GER') | df_players.ioc.eq('FRG')| df_players.ioc.eq('EUA') | df_players.ioc.eq('DEU')]
df_ger_players

,player_id,name_first,name_last,hand,dob,ioc,ioc_nation,height,wikidata_id
32,100033,Wilhelm,Bungert,R,19390401.0,GER,Germany,188.0,Q77579
33,100034,Christian,Kuhnke,L,19390413.0,GER,Germany,NaN,Q461558
55,100056,Harald,Elschenbroich,R,19410619.0,GER,Germany,NaN,Q5653740
84,100085,Hans Joachim,Ploetz,R,19440226.0,GER,Germany,NaN,NaN
136,100137,Hans Jurgen,Pohmann,R,19470523.0,GER,Germany,178.0,Q95218
...,...,...,...,...,...,...,...,...,...
58470,211820,Hans Joachim,Richter,U,NaN,GDR,East Germany,NaN,NaN
58539,211889,Gunter,Reimann,U,NaN,GER,Germany,NaN,NaN
58571,211921,Horst,Stahlberg,U,NaN,GDR,East Germany,NaN,NaN
58581,211931,Gunther,Sanders,U,NaN,GER,Germany,NaN,NaN


Für die Auswertung wird benötigt, ob ein deutscher Spieler überhaupt an einem Match teilgenommen hat.  
Dieses neue Feature braucht aber ein paar zwischenschritte bevor wir dieses erstellen können.  
Als ersten Schritt fügen wir ein Feature hinzu, das sagt ob ein deutscher das Match gewonnen hat.

In [7]:
df_matches['ger_win'] = df_matches.winner_id.isin(df_ger_players.player_id)
df_matches.sample(5)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ger_win
160883,2013-580,Australian Open,Hard,128,G,20130114,39,104330,NaN,Q,...,46.0,25.0,19.0,6.0,11.0,130.0,433.0,85.0,624.0,False
34114,1977-2008,Helsinki,Carpet,32,A,19770314,289,100236,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,NaN,False
153373,2010-540,Wimbledon,Grass,128,G,20100621,51,104770,NaN,Q,...,26.0,24.0,13.0,7.0,13.0,164.0,294.0,124.0,422.0,False
109003,1997-580,Australian Open,Hard,128,G,19970113,111,102446,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,36.0,1114.0,109.0,433.0,False
101801,1995-357,Stuttgart Indoor,Carpet,32,A,19950220,28,101990,NaN,NaN,...,57.0,25.0,16.0,4.0,5.0,16.0,1569.0,15.0,1587.0,False


Als zweiten Schitt fügen wir nun ein Feature hinzu, das sagt ob ein deutscher das Match verloren hat.

In [8]:
df_matches['ger_lose'] = df_matches.loser_id.isin(df_ger_players.player_id)
df_matches.sample(5)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ger_win,ger_lose
138102,2005-560,US Open,Hard,128,G,20050829,21,103970,17.0,NaN,...,38.0,23.0,14.0,25.0,15.0,1420.0,67.0,547.0,False,False
48470,1980-422,Cincinnati,Hard,64,M,19800818,17,100255,6.0,NaN,...,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,False,False
159582,2012-316,Bastad,Clay,28,A,20120709,6,105777,6.0,NaN,...,10.0,8.0,4.0,7.0,69.0,671.0,100.0,550.0,False,False
141828,2006-773,Bucharest,Clay,32,A,20060911,13,104252,7.0,NaN,...,5.0,8.0,1.0,6.0,53.0,705.0,101.0,403.0,True,False
164508,2014-403,Miami Masters,Hard,96,M,20140319,28,105173,NaN,NaN,...,7.0,11.0,2.0,6.0,80.0,656.0,71.0,690.0,False,False


Mit diesen 2 neuen Features lässt sich nun einfach ein 3 Feature erstellen, das als Flag dient ob überhaupt ein Deutscher teilgenommen hat am Match.

In [9]:
#Ein Deutscher hat teilgenommen, wenn der Sieger ein Deutscher war oder der Verlierer ein Deutscher war, oder auch beides
df_matches['ger_participated'] = df_matches['ger_win'] | df_matches['ger_lose']

In [10]:
df_matches.loc[df_matches.ger_participated.eq(True)].sample(5)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ger_win,ger_lose,ger_participated
90783,1992-717,Tampa,Clay,32,A,19920413,16,101736,2.0,NaN,...,9.0,3.0,6.0,17.0,1272.0,105.0,347.0,False,True,True
39367,1978-359,Bogota,Clay,32,A,19781119,30,100403,NaN,NaN,...,NaN,NaN,NaN,69.0,NaN,NaN,NaN,True,False,True
139866,2006-375,Lyon,Carpet,32,A,20061023,18,104597,8.0,NaN,...,11.0,0.0,1.0,42.0,845.0,55.0,705.0,False,True,True
162495,2013-540,Wimbledon,Grass,128,G,20130624,84,104460,NaN,Q,...,20.0,1.0,6.0,189.0,261.0,70.0,710.0,True,False,True
118509,1999-418,Washington,Hard,56,A,19990816,53,102338,1.0,NaN,...,12.0,7.0,11.0,2.0,3551.0,16.0,1884.0,False,True,True


Da bereits wie oben erwähnt der Davis Cup besonders Schwierigkeiten macht bei der location, werden diese nun jetzt entfernt.

In [11]:
df_matches = df_matches[~df_matches['tourney_name'].str.contains('Davis')]

In [12]:
df_matches

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,ger_win,ger_lose,ger_participated
0,1968-2029,Dublin,Grass,32,A,19680708,270,112411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
1,1968-2029,Dublin,Grass,32,A,19680708,271,126914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
2,1968-2029,Dublin,Grass,32,A,19680708,272,209523,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
3,1968-2029,Dublin,Grass,32,A,19680708,273,100084,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
4,1968-2029,Dublin,Grass,32,A,19680708,274,100132,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187948,2022-0605,Tour Finals,Hard,8,F,20221114,290,126094,6.0,NaN,...,13.0,2.0,5.0,7.0,3530.0,3.0,5350.0,False,False,False
187949,2022-0605,Tour Finals,Hard,8,F,20221114,289,104925,7.0,NaN,...,11.0,1.0,2.0,8.0,3320.0,3.0,5350.0,False,False,False
187950,2022-0605,Tour Finals,Hard,8,F,20221114,288,126094,6.0,NaN,...,17.0,6.0,9.0,7.0,3530.0,5.0,4065.0,False,False,False
187951,2022-0605,Tour Finals,Hard,8,F,20221114,287,104925,7.0,NaN,...,16.0,4.0,6.0,8.0,3320.0,5.0,4065.0,False,False,False


Es wird nun Gruppiert nach Turniername. Von besonderen Interesse sind je Turnier:  
* Wie viele Matches es insgeamt gibt
* Bei wie vielen Matches spielte ein Deutscher
* Wie viele Matches hatten die Deutschen gewonnen.

In [13]:
agg_df = df_matches.groupby('tourney_name').agg(total_games_in_tourney=('ger_participated', 'count'),
                                                total_games_with_ger=('ger_participated', 'sum'),
                                                wins=('ger_win', 'sum')).reset_index()


In [14]:
agg_df.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins
55,Berlin,372,193,104
86,Buenos Aires NTL,4,0,0
179,Grand Slam Cup,142,27,19
62,Boca Raton,62,9,7
366,Palma,47,4,1


Es wird dem DataFrame ein neues Feature hinzugefügt für die Siegchance.

In [15]:
agg_df['success_rate'] = agg_df['wins'] / agg_df['total_games_with_ger'] * 100
agg_df.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate
18,Aptos,31,0,0,NaN
442,Sofia,251,42,22,52.380952
207,Indian Wells,165,14,12,85.714286
100,Caracas WCT,43,0,0,NaN
424,Sao Paulo,526,28,10,35.714286


Zur Prüfung werden die Zeilen sortiert nach Siegrate und angezeigt.

In [16]:
agg_df.sort_values(by='success_rate', ascending=False)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate
136,Dallas WCT,151,3,3,100.000000
207,Indian Wells,165,14,12,85.714286
247,Laver Cup,40,7,6,85.714286
106,Charleston,77,6,5,83.333333
7,Aix-en-Provence,47,6,5,83.333333
...,...,...,...,...,...
523,Wembley 3,16,0,0,NaN
524,Wembley NTL 1,3,0,0,NaN
525,Wembley NTL 2,6,0,0,NaN
528,World Invitational Classic,15,0,0,NaN


Für die Fragestellung haben Turniere bei denen es nur wenig Spiele mit Deutschen gibt nicht wirklich relevant. Ein Turnier bei zum Beispiel 4 mal gespielt wurde und 4 mal gewonnen wurde hat mit seiner 100% Siegwahrscheinlichkeit eher eine geringe Aussage.  
  
Da es viele Turniere gibt mit nur kleiner Teilnehmeranzahl, wird das 90% Quantil genutzt des Features total_games_with_ger(Anzahl Matches mit Deutschen).

In [17]:
quantile_90 = agg_df['total_games_with_ger'].quantile(0.90)
quantile_90

91.5

Das 90% Quantil wird als Filter benutzt und es wird sortiert nach der Siegrate.

In [18]:
filtered_df = agg_df.loc[agg_df['total_games_with_ger'] > quantile_90]
filtered_df.sort_values(by='success_rate', ascending = False)


,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate
81,Brussels,635,93,60,64.516129
306,Milan,744,96,60,62.500000
209,Indianapolis,2375,159,93,58.490566
391,Queen's Club,2521,175,101,57.714286
184,Halle,887,388,223,57.474227
451,St. Petersburg,912,151,84,55.629139
506,Vienna,1552,265,146,55.094340
319,Moscow,917,143,78,54.545455
481,Tokyo,1187,182,99,54.395604
294,Memphis,1686,159,86,54.088050


Für die Fragestellungen sind hier jetzt relevante Turniere bei denen die Deutschen gut oder eher schlecht abschneiden.  Die atp_tourneys Tabelle ist angereichert worden mit Koordinaten. Diese werden benötigt für eine Geographische Darstellung.

In [19]:
df_coords = pd.read_csv('data/atp_tourneys_till_2022_coordinates.csv')
df_coords.sample(5)

,tourney_id,tourney_name,tourney_location,surface,draw_size,tourney_level,tourney_date,latitude,longitude
2541,1992-D047,Davis Cup G2 SF: LUX vs GRE,Davis Cup G2 SF: LUX vs GRE,Carpet,4,D,19920925,NaN,NaN
7633,1975-428,Tokyo Indoor,Tokyo,Carpet,32,A,19750414,35.764784,139.148599
1054,1996-499,Coral Springs,Coral Springs,Clay,32,A,19960513,26.271192,-80.270604
1938,1984-D033,Davis Cup EUR R1: MAR vs FIN,Davis Cup EUR R1: MAR vs FIN,Clay,4,D,19840504,NaN,NaN
1622,1977-D016,Davis Cup EUR SF: RSA vs COL,Davis Cup EUR SF: RSA vs COL,NaN,4,D,19771215,NaN,NaN


In [20]:
#Mergen für die Koordinaten
df_finished = pd.merge(filtered_df, df_coords[['tourney_name','tourney_location','latitude','longitude']], on='tourney_name',how='left').drop_duplicates(subset='tourney_name').sort_values(by='success_rate', ascending = False)

Die Fragestellung spricht von "... besonders gut oder besonders schlecht
abschneiden." Dafür haben Grenzen gesetzt je nach Siegrate:
* Ab über 55% Siegrate gelten die deutschen als sehr stark.
* Zwischen 55% und 52% als stark.
* Zwischen 48% und 45% als schwach
* Ab unter 45 als sehr schwach

In [21]:
df_ger_strong = df_finished.loc[df_finished.success_rate.gt(52) & df_finished.success_rate.le(55)]
df_ger_strong.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate,tourney_location,latitude,longitude
974,Memphis,1686,159,86,54.088050,Memphis,35.149022,-90.051629
1128,Moscow,917,143,78,54.545455,Moscow,55.750446,37.617494
1429,Rome Masters,1950,184,99,53.804348,Rome,41.893320,12.482932
296,Berlin,372,193,104,53.886010,Berlin,52.517037,13.388860
1603,Stuttgart Outdoor,937,295,156,52.881356,Stuttgart,48.778449,9.180013


In [22]:
df_ger_very_strong = df_finished.loc[df_finished.success_rate.gt(55)]
df_ger_very_strong.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate,tourney_location,latitude,longitude
307,Brussels,635,93,60,64.516129,Brussels,50.846557,4.351697
1072,Milan,744,96,60,62.500000,Milan,45.463945,9.188558
632,Halle,887,388,223,57.474227,Halle,48.862204,2.347668
1327,Queen's Club,2521,175,101,57.714286,London,51.507322,-0.127647
1508,St. Petersburg,912,151,84,55.629139,St. Petersburg,27.770380,-82.669509


In [23]:
df_ger_weak = df_finished.loc[df_finished.success_rate.lt(48) & df_finished.success_rate.ge(45)]
df_ger_weak.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate,tourney_location,latitude,longitude
1884,s Hertogenbosch,685,112,52,46.428571,s Hertogenbosch,51.688935,5.303104
772,Indian Wells Masters,2512,279,131,46.953405,Indian Wells,33.721797,-116.338303
1262,Nice,1047,98,45,45.918367,Nice,43.710082,7.261844
954,Madrid Masters,1044,94,45,47.872340,Madrid,40.416705,-3.703583
1537,Stockholm,1763,193,88,45.595855,Stockholm,59.325117,18.071093


In [24]:
df_ger_very_weak = df_finished.loc[df_finished.success_rate.lt(45)]
df_ger_very_weak.sample(5)

,tourney_name,total_games_in_tourney,total_games_with_ger,wins,success_rate,tourney_location,latitude,longitude
243,Bastad,1611,98,36,36.734694,Bastad,56.426790,12.860715
1374,Roland Garros,6937,707,314,44.413013,Roland Garros,48.847300,2.248355
141,Barcelona,3081,261,117,44.827586,Barcelona,41.382894,2.177432
1655,Tokyo Outdoor,1093,98,43,43.877551,Tokyo,35.764784,139.148599
1462,Rotterdam,1409,172,73,42.441860,Rotterdam,51.924442,4.477750


Jetzt sind alle DataFrames vorhanden mit allen benötigten Features. Mithilfe vom Modul folium werden die Turniere nun auf eine Karte platziert.

In [25]:
map = folium.Map(location=[df_ger_strong['latitude'].mean(), df_ger_strong['longitude'].mean()], zoom_start=4)

for index, row in df_ger_very_strong.iterrows():
    icon = folium.Icon(color='green', icon='info-sign')
    folium.Marker([row['latitude'], row['longitude']], popup=row['tourney_name'], icon=icon).add_to(map)
    
for index, row in df_ger_strong.iterrows():
    icon = folium.Icon(color='lightgreen', icon='info-sign')
    folium.Marker([row['latitude'], row['longitude']], popup=row['tourney_name'], icon=icon).add_to(map)
    
for index, row in df_ger_weak.iterrows():
    icon = folium.Icon(color='orange', icon='info-sign')
    folium.Marker([row['latitude'], row['longitude']], popup=row['tourney_name'], icon=icon).add_to(map)
    
for index, row in df_ger_very_weak.iterrows():
    icon = folium.Icon(color='red', icon='info-sign')
    folium.Marker([row['latitude'], row['longitude']], popup=row['tourney_name'], icon=icon).add_to(map)
map

Die Karte zeigt alle Turniere bei denen die Deutschen (sehr) stark oder (sehr) schwach waren(in Bezug auf die Siegrate).  
Es gilt:
* Dunkelgrün: Deutsche sind hier sehr stark
* Grün: Deutsche sind hier stark
* Orange: Deutsche sind hier schwach
* Rot: Deutsche sind hier sehr schwach

Folium erlaubt die Verwendung von MarkerClustern und Layern. Hier kann nach belieben nur bestimmte Cluster getoggled werden und beim rauszoomen werden nahe Markierungen zusammengefasst.

In [26]:
map = folium.Map(location=[df_ger_strong['latitude'].mean(), df_ger_strong['longitude'].mean()], zoom_start=4)

# Funktion zum Hinzufügen von Markern und Popup-Informationen
def add_markers(df, color, cluster):
    for index, row in df.iterrows():
        icon = folium.Icon(color=color, icon='info-sign')
        folium.Marker([row['latitude'], row['longitude']], popup=row['tourney_name'], icon=icon).add_to(cluster)

# Erstelle Marker-Cluster
cluster_very_strong = plugins.MarkerCluster(name='Very Strong Markers').add_to(map)
cluster_strong = plugins.MarkerCluster(name='Strong Markers').add_to(map)
cluster_weak = plugins.MarkerCluster(name='Weak Markers', color='orange').add_to(map)
cluster_very_weak = plugins.MarkerCluster(name='Very Weak Markers').add_to(map)

# Füge Marker zu den Clustern hinzu
add_markers(df_ger_very_strong, 'green', cluster_very_strong)
add_markers(df_ger_strong, 'lightgreen', cluster_strong)
add_markers(df_ger_weak, 'orange', cluster_weak)
add_markers(df_ger_very_weak, 'red', cluster_very_weak)

# Füge Marker-Cluster zur Karte hinzu
cluster_very_strong.add_to(map)
cluster_strong.add_to(map)
cluster_very_weak.add_to(map)
cluster_weak.add_to(map)

# Füge Layer Control zur Karte hinzu, um zwischen den Clustern zu wechseln
folium.LayerControl().add_to(map)
map

Für die Fragestellung lässt sich sagen, dass Deutschland in Turnieren innerhalb Europa und Amerika sehr große Schwankungen in ihrer Siegrate aufweist. Deutschland ist hier entweder besonders gut oder besonders schlecht je nach Turnier. Verblüffender Weise ist Deutschland in Amerika öfters stärker als schwächer, in Europa hingegen öfters schwächer als stärker. Für Asien(Japan) und Australien ist Deutschland jeweils sehr schwach bzw. schwach.